<a href="https://colab.research.google.com/github/cgangwar11/Coding-Practice/blob/master/demo_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import spacy
nlp = spacy.load('en')
from pprint import  pprint

In [3]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive


In [0]:
import pickle
def load_vocab():

  with open('/gdrive/My Drive/stoi.pkl','rb') as f:
    stoi,itos = pickle.load(f)
  return stoi,itos

In [0]:
import torch.nn as nn
import torch.nn.functional as F
class CNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim, dropout):
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.convs = nn.ModuleList([nn.Conv2d(in_channels=1, out_channels=n_filters, kernel_size=(fs,embedding_dim)) for fs in filter_sizes])
        self.fc = nn.Linear(len(filter_sizes)*n_filters, output_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        
        #x = [sent len, batch size]
        
        x = x.permute(1, 0)
                
        #x = [batch size, sent len]
        
        embedded = self.embedding(x)
                
        #embedded = [batch size, sent len, emb dim]
        
        embedded = embedded.unsqueeze(1)
        
        #embedded = [batch size, 1, sent len, emb dim]
        
        conved = [F.relu(conv(embedded)).squeeze(3) for conv in self.convs]
            
        #conv_n = [batch size, n_filters, sent len - filter_sizes[n]]
        
        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]
        
        #pooled_n = [batch size, n_filters]
        
        cat = self.dropout(torch.cat(pooled, dim=1))

        #cat = [batch size, n_filters * len(filter_sizes)]
            
        return self.fc(cat)



In [0]:
stoi,itos = load_vocab()
INPUT_DIM = len(itos)
EMBEDDING_DIM = 100
N_FILTERS = 100
FILTER_SIZES = [3,4,5]
OUTPUT_DIM = 1
DROPOUT = 0.5


In [0]:

model = CNN(INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZES, OUTPUT_DIM, DROPOUT)

In [0]:
model.load_state_dict(torch.load('/gdrive/My Drive/basic_9_epochs_87_128_batch.pth'))

In [9]:
model

CNN(
  (embedding): Embedding(25002, 100)
  (convs): ModuleList(
    (0): Conv2d(1, 100, kernel_size=(3, 100), stride=(1, 1))
    (1): Conv2d(1, 100, kernel_size=(4, 100), stride=(1, 1))
    (2): Conv2d(1, 100, kernel_size=(5, 100), stride=(1, 1))
  )
  (fc): Linear(in_features=300, out_features=1, bias=True)
  (dropout): Dropout(p=0.5)
)

In [0]:


def predict_sentiment(sentence, min_len=5):
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    if len(tokenized) < min_len:
        tokenized += ['<pad>'] * (min_len - len(tokenized))
    print(tokenized)
    indexed = []
    for t in tokenized:
      if t in stoi.keys():
        indexed.append(stoi[t])
      else:
        indexed.append(0)
    print(' '.join([itos[i] for i in indexed]))
    tensor = torch.LongTensor(indexed)
    tensor = tensor.unsqueeze(1)
    
    prediction = torch.sigmoid(model(tensor))
    return prediction.item()

In [11]:
model.eval()

CNN(
  (embedding): Embedding(25002, 100)
  (convs): ModuleList(
    (0): Conv2d(1, 100, kernel_size=(3, 100), stride=(1, 1))
    (1): Conv2d(1, 100, kernel_size=(4, 100), stride=(1, 1))
    (2): Conv2d(1, 100, kernel_size=(5, 100), stride=(1, 1))
  )
  (fc): Linear(in_features=300, out_features=1, bias=True)
  (dropout): Dropout(p=0.5)
)

In [0]:
from fastai.text import *
path = untar_data(URLs.IMDB_SAMPLE)
df = pd.read_csv(path/'texts.csv')

In [38]:
predict_sentiment(df.text[6])

['There', 'were', 'a', 'lot', 'of', 'truly', 'great', 'horror', 'movies', 'produced', 'in', 'the', 'seventies', '-', 'but', 'this', 'film', 'certainly', 'is', "n't", 'one', 'of', 'them', '!', 'It', "'s", 'a', 'shame', 'The', 'Child', 'is', "n't", 'better', 'as', 'it', 'works', 'from', 'a', 'decent', 'idea', 'that', 'takes', 'in', 'a', 'couple', 'of', 'sometimes', 'successful', 'horror', 'themes', '.', 'We', 'have', 'the', 'idea', 'of', 'a', 'vengeful', 'child', ',', 'which', 'worked', 'so', 'well', 'in', 'classic', 'films', 'such', 'as', 'The', 'Bad', 'Seed', 'and', 'then', 'we', 'have', 'the', 'central', 'zombie', 'theme', ',', 'which', 'of', 'course', 'has', 'been', 'the', 'backbone', 'of', 'many', 'a', 'successful', 'horror', 'movie', '.', 'The', 'plot', 'is', 'basically', 'this', ':', 'young', 'girl', 'blames', 'a', 'load', 'of', 'people', 'for', 'the', 'death', 'of', 'her', 'mother', ',', 'so', 'she', 'goes', 'to', 'the', 'graveyard', 'and', 'raises', 'the', 'dead', 'to', 'get', '

0.14498916268348694

In [0]:
corpus = """The start of 2019 has been an intense time for all of us. And I am proud of all the effort that so many of you have put in. We have started the new year with good momentum in a lot of areas. It is now vital that we work hard to keep that momentum going across our entire company. And I know that I can count on you to put your hearts and souls into our joint success. 
 
To further strengthen our client focus and enhance shareholder value, we have to become a more effective organization. We have been stressing the importance of being efficient and agile over the last couple of months. After initial changes to the Executive Board and our regional leadership structure in November and December, the Executive Board has now mutually agreed, following a vigorous process and intense discussions over the last weeks, to make several organizational changes, determining the home of best fit for various units, introducing leaner structures with clear responsibilities, and ending parallel work streams.

The following changes will go into effect globally on 1 March:

•	Global Trading is part of our investment process and will move from Operations to the Global Investment Group.

•	The Investment Specialists will be transferred to the Global Coverage Group as they are closely linked to our sales efforts. Product Strategy & Analytics will remain in the Global Investment Group.

•	The Business Development/RfP Team will split from the Client Implementation Group within COO and transfer to the Global Coverage Group.

•	The Illiquid Operations team will move from COO to the Alternatives arm of the Global Investment Group.

•	To strengthen our first line of defense, we are re-establishing the DCO (Divisional Control Office) function under the leadership of the COO.

•	Finally, we are forming a Global Operating Committee, chaired by Mark Cullen, which will ensure we are aligned across DWS in managing all operational topics, including the execution of cost optimization measures.

In executing the above, further changes will be triggered within some areas. The respective Board members will inform you about any changes within their units.

I cannot emphasize enough how important a phase we are currently in. It is vital that we all pull in the same direction and keep our positive momentum going. Whatever changes might be necessary and whatever obstacles we might face, we can never lose sight of what counts most: our fiduciary duty to our clients and through that the delivery of value to our shareholders.

We as DWS are capable of extraordinary things. So let us continue to work closely together in doing what we do best, and I am confident that we will be successful in 2019 and beyond.

Be Inspiring. Be Hard Working. Be Proud. Be One DWS.
"""

In [0]:
corpus2 = """We have passed the biggest tax cuts and reforms in American history. We’ve started the construction of a major border wall, and we have greatly strengthened border security.

We have secured record funding for our military -- $700 billion this year, and $716 billion next year. Our military will soon be more powerful than it has ever been before.

In other words, the United States is stronger, safer, and a richer country than it was when I assumed office less than two years ago.

We are standing up for America and for the American people. And we are also standing up for the world.

This is great news for our citizens and for peace-loving people everywhere. We believe that when nations respect the rights of their neighbors, and defend the interests of their people, they can better work together to secure the blessings of safety, prosperity, and peace.

Each of us here today is the emissary of a distinct culture, a rich history, and a people bound together by ties of memory, tradition, and the values that make our homelands like nowhere else on Earth.

That is why America will always choose independence and cooperation over global governance, control, and domination.

I honor the right of every nation in this room to pursue its own customs, beliefs, and traditions. The United States will not tell you how to live or work or worship. We only ask that you honor our sovereignty in return.

From Warsaw to Brussels, to Tokyo to Singapore, it has been my highest honor to represent the United States abroad. I have forged close relationships and friendships and strong partnerships with the leaders of many nations in this room, and our approach has already yielded incredible change.

With support from many countries here today, we have engaged with North Korea to replace the specter of conflict with a bold and new push for peace.

In June, I traveled to Singapore to meet face to face with North Korea’s leader, Chairman Kim Jong Un. We had highly productive conversations and meetings, and we agreed that it was in both countries’ interest to pursue the denuclearization of the Korean Peninsula. Since that meeting, we have already seen a number of encouraging measures that few could have imagined only a short time ago.

The missiles and rockets are no longer flying in every direction. Nuclear testing has stopped. Some military facilities are already being dismantled. Our hostages have been released. And as promised, the remains of our fallen heroes are being returned home to lay at rest in American soil.

I would like to thank Chairman Kim for his courage and for the steps he has taken, though much work remains to be done. The sanctions will stay in place until denuclearization occurs.

I also want to thank the many member states who helped us reach this moment -- a moment that is actually far greater than people would understand; far greater -- but for also their support and the critical support that we will all need going forward.

A special thanks to President Moon of South Korea, Prime Minister Abe of Japan, and President Xi of China."""

In [0]:
corpus = corpus2.lower()

In [0]:
doc = nlp(corpus)

In [76]:
k=[]
for i in doc.sents:
    k.append((predict_sentiment(i.text),i.text))

['we', 'have', 'passed', 'the', 'biggest', 'tax', 'cuts', 'and', 'reforms', 'in', 'american', 'history', '.']
we have passed the biggest tax cuts and <unk> in american history .
['we', '’ve', 'started', 'the', 'construction', 'of', 'a', 'major', 'border', 'wall', ',', 'and', 'we', 'have', 'greatly', 'strengthened', 'border', 'security', '.', '\n\n']
we <unk> started the construction of a major border wall , and we have greatly <unk> border security . <unk>
['we', 'have', 'secured', 'record', 'funding', 'for', 'our', 'military', '--', '$', '700', 'billion', 'this', 'year', ',', 'and', '$', '716', 'billion', 'next', 'year', '.']
we have <unk> record funding for our military -- $ 700 billion this year , and $ <unk> billion next year .
['our', 'military', 'will', 'soon', 'be', 'more', 'powerful', 'than', 'it', 'has', 'ever', 'been', 'before', '.', '\n\n']
our military will soon be more powerful than it has ever been before . <unk>
['in', 'other', 'words', ',', 'the', 'united', 'states', 'i

In [0]:
positive=[]
negative=[]
neutral = []
p_t,n_t=0.9,0.3
for i,j in k:
    if i>=p_t:
        positive.append(j)
    elif i<n_t:
        negative.append(j)
    else:
        neutral.append(j)

In [0]:
positive_words = ' . '.join(positive)
negative_words = ' . '.join(negative)
neutral_words = ' . '.join(neutral)

In [0]:
def print_list(a):
    for i in a:
        print(i)

In [86]:
print_list(positive)

our military will soon be more powerful than it has ever been before.


and we are also standing up for the world.


this is great news for our citizens and for peace-loving people everywhere.
i have forged close relationships and friendships and strong partnerships with the leaders of many nations in this room, and our approach has already yielded incredible change.

with support from many countries here today, we have engaged with north korea to replace the specter of conflict with a bold and new push for peace.


i also want to thank the many member states who helped us reach this moment -- a moment that is actually far greater than people would understand; far greater -- but for also their support and the critical support that we will all need going forward.




In [87]:
print_list(negative)

we’ve started the construction of a major border wall, and we have greatly strengthened border security.


from warsaw to brussels, to tokyo to singapore, it has been my highest honor to represent the united states abroad.
in june, i traveled to singapore to meet face to face with north korea’s leader, chairman kim jong un.
since that meeting, we have already seen a number of encouraging measures that few could have imagined only a short time ago.


some military facilities are already being dismantled.
our hostages have been released.
a special thanks to president moon of south korea, prime minister abe of japan, and president xi of china.


In [81]:
pprint(neutral_words)

('we have passed the biggest tax cuts and reforms in american history. . we’ve '
 'started the construction of a major border wall, and we have greatly '
 'strengthened border security.\n'
 '\n'
 ' . we have secured record funding for our military -- $700 billion this '
 'year, and $716 billion next year. . in other words, the united states is '
 'stronger, safer, and a richer country than it was when i assumed office less '
 'than two years ago.\n'
 '\n'
 ' . we are standing up for america and for the american people. . we believe '
 'that when nations respect the rights of their neighbors, and defend the '
 'interests of their people, they can better work together to secure the '
 'blessings of safety, prosperity, and peace.\n'
 '\n'
 ' . each of us here today is the emissary of a distinct culture, a rich '
 'history, and a people bound together by ties of memory, tradition, and the '
 'values that make our homelands like nowhere else on earth.\n'
 '\n'
 ' . that is why america will 